In [99]:
###############################################################################
# KGB user input vars  (Preferably implement as separate upstream cell)
###############################################################################        


################################################################################################
# Config #1: local notebook with NCBI annotated genbank format genomes and gene tree           #
#            Domain Annotations: NO (KBase domain annotations require KBase genome annotation) #
#            Gene Tree: YES                                                                    #
#            Search Terms: YES                                                                 #
################################################################################################
                                                                                             
%pylab notebook
KBase_backend = False

GenomeSet_names = ["Gsulf", "DvulH", "DdesulfG20", "EcoliK12", "Bsub", "DaudaxMP104C"]
ContigSet_names = []
PivotFeatures_IDs = ["GSU2863", "DVU2928", "Dde_2997", "b3987", "BSU01070", "Daud0216"]
PrimaryAnchor_leafId = "Gsulf rpoB"
PrimaryAnchor_locusTag = "GSU2863"

genome_annotation_system = 'NCBI'
genome_data_format = "Genbank"
# if relative path to scaffolds is e.g.: ./data_example/NCBI_genome_contigs/Gsulf/scaffolds/scaf_1.gbk
gbk_ext = "gbk"
genome_data_base_path = "./data_example/NCBI_genome_contigs"
genome_data_extra_subpath = "/scaffolds"

domain_data_exists = False  # NCBI annotated genome
domain_data_format = None
domain_data_base_path = None
domain_data_extra_subpath = None
domain_family_desc_base_path = None

tree_data_format = 'newick'
# if relative path to tree is e.g.: ./data_example/trees/rpoB_tree-names.newick
tree_data_base_path = './data_example/trees'
tree_data_file = 'rpoB_tree-names.newick'

Search_Terms = ['dna-directed polymerase',
                '16S',
                'DNA and methyltransferase',
                '1.10.3.-',
                'fucI',
                'sulfate adenylyl transferase']

from urllib.request import urlopen
import_code = urlopen('https://raw.githubusercontent.com/dcchivian/KGB/master/KGB.py')
exec(import_code.read())

"""
##########################################################################
# Config #1: local notebook with KBase annotated genbank format plasmids #
#            as separate contig gbk files in a directory for each sample #
#            Domain Annotations: YES                                     #
#            Gene Tree: NO                                               #
##########################################################################

%pylab notebook
KBase_backend = False

GenomeSet_names = ["GW456", "GW460"]
ContigSet_names = []
PivotFeatures_IDs = []
PrimaryAnchor_leafId = "GW456.scaffold_130"
PrimaryAnchor_locusTag = "GW456_SEED_annot.CDS.1"

genome_annotation_system = 'KBase'
genome_data_format = "Genbank"
# if relative path to scaffolds is e.g.: ../data/plasmid_contigs/GW456_old/scaffolds/scaffold_130.gbk
gbk_ext = "gbk"
genome_data_base_path = "../data/plasmid_contigs"
genome_data_extra_subpath = "_old/scaffolds"
#genome_data_extra_subpath = "_circular_2KbpMIN/scaffolds"

domain_data_exists = True
domain_data_format = "KBase_domains"
# if relative path to domain annotations is e.g.: ../data/plasmid_contigs/GW456_old/domains/GW456_Domain_annot.json
domain_data_base_path = "../data/plasmid_contigs"
domain_data_extra_subpath = "_old/domains"
domain_family_desc_base_path = "../data/domain_desc"

tree_data_format = 'newick'
tree_data_base_path = None
tree_data_file = None

#Search_Terms = []
Search_Terms = ['pil or conj or mobil or partition or t-dna or vir or plasmid',
                'toxin',
                'tra',
                'rep'
               ]
"""
"""
# KBase genomes in KBase context
#
#KBase_backend = True
#genome_data_format = "KBase"
#GenomeSet_ref = '16750/58/1'
#GenomeSet_refs = []
#GenomeSet_refs = ['16750/7/1',
#                   '16750/8/2',
#                   '16750/9/1',
#                   '16750/10/1',
#                   '16750/11/1'
#                   ]
#GenomeSet_names = []
#PivotFeatures_IDs = []
#Search_Terms = []
#domain_data_exists = True
#domain_data_format = "KBase_domains"
#domain_data_base_path = None
#domain_family_desc_base_path = None
#tree_data_format = 'newick'
#tree_data_base_path = None
#tree_data_file = None
#PrimaryAnchor_leafId = None
#PrimaryAnchor_locusTag = None


# KBase genomes
#
#ref='ReferenceGenomeAnnotations/kb|g.3899'
#ref='ReferenceGenomeAnnotations/kb|g.2624'
#ref='ReferenceGenomeAnnotations/kb|g.2424'
#GenomeSet_names = ['ReferenceGenomeAnnotations/kb|g.2424']




# KBase Genbank genomes
#
#GenomeSet_names = ["Desulforudis_audaxviator",
#                   "Pelotomaculum_thermopropionicum",
#                   "Desulfotomaculum_reducens",
#                   "Bacillus_subtilis",
#                   "Desulfovibrio_desulfuricans",
#                   "Desulfovibrio_vulgaris",
#                   "Geobacter_sulfurreducens",
#                   "Escherichia_coli"]
##                   "Desulforudis_audaxviator",
##                   "Bacillus_subtilis",
#                   #"Moorella_thermoacetica",
#                   #"Thermoanaerobacter_tengcongensis",
#
#ContigSet_names = []
#PivotFeatures_IDs = ["kb|g.1713.peg.340",
#                         "kb|g.1713.peg.368",
#                         "kb|g.1106.peg.1477",
#                         "kb|g.996.peg.3228",
#                         "kb|g.423.peg.584",
#                         "kb|g.423.peg.176",
#                         "kb|g.357.peg.1850",
#                         "kb|g.3562.peg.757",
#                         "kb|g.483.peg.109",
#                         "kb|g.26833.CDS.2562"]
#PrimaryAnchor_leafId = "Daudaxviator-Sat"
#PrimaryAnchor_locusTag = "kb|g.1713.peg.340"
#
#genome_annotation_system = 'KBase'
#genome_data_format = "Genbank"
#genome_data_base_path = "./data/KBase_genome_contigs"
#genome_data_extra_subpath = ""
##genome_data_extra_subpath = "/scaffolds"
#
#domain_data_format = "KBase_domains"
#domain_data_base_path = None
#domain_data_base_path = "./data/KBase_genome_contigs"
#domain_data_extra_subpath = ""
##domain_data_extra_subpath = "/domains"
#domain_family_desc_base_path = "./data/domain_desc"
#
#tree_data_format = 'newick'
#tree_data_base_path = None
#tree_data_file = None
##tree_data_base_path = './data/trees'
##tree_data_file = 'rpoB_tree-names.newick'
#
##Search_Terms = []
#Search_Terms = ['dna-directed polymerase',
#                '16S',
#                'DNA and methyltransferase',
#                '1.10.3.-',
##                'polymerase or dna-directed',
#                'fucI',
#                'sulfate adenylyl transferase']
"""





Populating the interactive namespace from numpy and matplotlib


SyntaxError: from __future__ imports must occur at the beginning of the file (<string>, line 79)